In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity_operationalizing")
exp = Experiment(workspace=ws, name="udacity-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Udacity_operationalizing
Azure region: eastus
Subscription id: 6d96438c-1dc7-4ef5-892c-3155ca1be569
Resource group: Udacity


In [70]:
# adapted from the auto-ml-classification-bank-marketing-all-features.ipynb example notebook
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4,
                                                           vm_priority='lowpriority',
                                                           idle_seconds_before_scaledown=900)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [119]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
parameter_space = {'C': uniform(0.1, 100)}
ps = RandomParameterSampling(parameter_space=parameter_space)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1)

params = {'--max_iter': 100, '--C' : 1}
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./train',
                entry_script='train.py',
                compute_target=compute_target,#'local'
                script_params=params)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                    hyperparameter_sampling=ps,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    max_duration_minutes=5,
                                    estimator=est,
                                    policy=policy
                                    )

In [120]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment

experiment = Experiment(ws, "logistic_regresssion_model")
run = experiment.submit(config=hyperdrive_config, show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C'] is the list of overridden parameter(s).


In [132]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [122]:
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
best_run.download_files(prefix='outputs')

In [123]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
logistic_regresssion_model,HD_39b890cf-50b0-4c63-98f9-bd11c29a3c36_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [133]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [137]:
import sys
sys.path.append('train')
from train import clean_data

# Use the clean_data function to clean the data.
x, y = clean_data(ds)

In [138]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=42)

In [169]:
import numpy as np
import pandas as pd
tr = np.concatenate((x_train, y_train[:, np.newaxis]), axis=-1)
tr_df = pd.DataFrame(data=tr, columns=x.columns.to_list() + ['y'])
tr_df.to_csv('cleaned_data.csv', index=False)
# now download file and re-upload to the datasets...this is super janky and obviously not best practices

In [182]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6d96438c-1dc7-4ef5-892c-3155ca1be569'
resource_group = 'Udacity'
workspace_name = 'Udacity_operationalizing'

workspace = Workspace(subscription_id, resource_group, workspace_name)

tr_ds = Dataset.get_by_name(workspace, name='cleaned_data.csv')

# tr_ds = TabularDatasetFactory.from_delimited_files('')

In [183]:
import azureml.train.automl.utilities
azureml.train.automl.utilities.get_primary_metrics('classification')

['norm_macro_recall',
 'precision_score_weighted',
 'average_precision_score_weighted',
 'AUC_weighted',
 'accuracy']

In [184]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

# can also set compute_target to run on cluster
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=tr_ds,
    label_column_name='y',
    n_cross_validations=3
    )

In [185]:
# Submit automl run
experiment = Experiment(ws, "automl")
run = experiment.submit(config=automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_3cc510e5-98d7-49ca-826f-80b2f8ec30d5

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [189]:
# Retrieve and save best automl model.
best_run, best_model = run.get_output()
joblib.dump(best_model, 'best_automl_model.joblib')

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



['best_automl_model.joblib']

In [192]:
# it was already deleted so it threw an error
try:
    compute_target.delete()
except:
    print('already deleted')

already deleted
